# Cellpose Interactive Demo

**Author:** Janan Arslan  
**Date:** 2025-06-03  
**License:** CC BY-NC-ND 4.0

This notebook provides an interactive demonstration of Cellpose for cell segmentation.

## Contents
1. Installation
2. Loading sample images
3. Basic segmentation
4. Comparing models
5. Parameter exploration
6. Visualizing results

## 1. Installation

First, let's install Cellpose. This might take a few minutes.

In [ ]:
# Install Cellpose (without GUI for Binder compatibility)
!pip install cellpose --quiet
!pip install matplotlib --quiet
!pip install ipywidgets --quiet

print("Installation complete!")

## 2. Import Libraries and Check Version

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from cellpose import models, io, plot, utils
from cellpose.io import get_image_files
import os
from urllib.request import urlretrieve
import ipywidgets as widgets
from IPython.display import display, clear_output

# Check Cellpose version - different methods
try:
    import cellpose
    # Try different ways to get version
    if hasattr(cellpose, '__version__'):
        print(f"Cellpose version: {cellpose.__version__}")
    else:
        # Alternative method
        import pkg_resources
        version = pkg_resources.get_distribution("cellpose").version
        print(f"Cellpose version: {version}")
except:
    print("Cellpose is installed but version info not available")
    
print("✓ All libraries imported successfully!")

## 3. Load Sample Images

Cellpose comes with built-in test images. Let's download and use them.

In [ ]:
# Load local images
image_files = {
    'cells': './Cell_Colony.jpg',
    'dots': './Dot_Blot.jpg',
    'aupbsn': './AuPbSn40.jpg'
}

# Load images
images = {}
for name, filepath in image_files.items():
    try:
        img = io.imread(filepath)
        images[name] = img
        print(f"✓ Loaded {name} image from {filepath}: shape {img.shape}")
    except FileNotFoundError:
        print(f"⚠ Could not find {filepath}")
    except Exception as e:
        print(f"⚠ Error loading {filepath}: {e}")

# If no images found, create synthetic data
if len(images) == 0:
    print("\nNo images found. Creating synthetic test image...")
    # Create a synthetic image with circles
    img_size = 512
    img = np.zeros((img_size, img_size, 3), dtype=np.uint8)
    
    # Add some circular "cells" without opencv
    for _ in range(20):
        # Create circular cells using numpy
        y, x = np.ogrid[-img_size:img_size, -img_size:img_size]
        cx = np.random.randint(50, img_size-50)
        cy = np.random.randint(50, img_size-50)
        radius = np.random.randint(20, 40)
        mask = (x-cx)**2 + (y-cy)**2 <= radius**2
        
        # Random color
        color = [np.random.randint(100, 255) for _ in range(3)]
        for c in range(3):
            img[:, :, c][mask[img_size:, img_size:]] = color[c]
    
    images['synthetic'] = img
    print("✓ Created synthetic test image")
    
# Convert grayscale images to RGB if needed
for name, img in images.items():
    if len(img.shape) == 2:
        # Convert grayscale to RGB
        images[name] = np.stack([img, img, img], axis=-1)
        print(f"  Converted {name} from grayscale to RGB")

print(f"\n✓ Loaded {len(images)} images successfully!")

In [ ]:
# Option 1: Load predefined local images
image_files = {
    'cells': './Cell_Colony.jpg',
    'dots': './Dot_Blot.jpg', 
    'aupbsn': './AuPbSn40.jpg'
}

# Load images
images = {}
for name, filepath in image_files.items():
    try:
        img = io.imread(filepath)
        images[name] = img
        print(f"✓ Loaded {name} image from {filepath}: shape {img.shape}")
    except FileNotFoundError:
        print(f"⚠ Could not find {filepath}")
    except Exception as e:
        print(f"⚠ Error loading {filepath}: {e}")

# Option 2: Add file upload widget for custom images
from ipywidgets import FileUpload
from IPython.display import display

upload_widget = FileUpload(
    accept='image/*',
    multiple=True,
    description='Upload Images:'
)

def on_upload_change(change):
    """Handle uploaded files"""
    for filename, file_info in upload_widget.value.items():
        content = file_info['content']
        # Convert bytes to numpy array
        import io as iolib
        from PIL import Image
        img = Image.open(iolib.BytesIO(content))
        img_array = np.array(img)
        
        # Add to images dict
        clean_name = filename.split('.')[0].lower().replace(' ', '_')
        images[clean_name] = img_array
        print(f"✓ Uploaded {filename} as '{clean_name}': shape {img_array.shape}")

upload_widget.observe(on_upload_change, names='value')

print("You can also upload your own images using the widget below:")
display(upload_widget)

# Convert grayscale images to RGB if needed
for name, img in list(images.items()):
    if len(img.shape) == 2:
        # Convert grayscale to RGB
        images[name] = np.stack([img, img, img], axis=-1)
        print(f"  Converted {name} from grayscale to RGB")

# If no images loaded, create synthetic data
if len(images) == 0:
    print("\nNo images found. Creating synthetic test image...")
    # Create a synthetic image with circles
    img_size = 512
    img = np.zeros((img_size, img_size, 3), dtype=np.uint8)
    
    # Add some circular "cells" using numpy only
    for _ in range(20):
        y, x = np.ogrid[-img_size:img_size, -img_size:img_size]
        cx = np.random.randint(50, img_size-50)
        cy = np.random.randint(50, img_size-50)
        radius = np.random.randint(20, 40)
        mask = (x-cx)**2 + (y-cy)**2 <= radius**2
        
        # Random color
        color = [np.random.randint(100, 255) for _ in range(3)]
        for c in range(3):
            img[:, :, c][mask[img_size:, img_size:]] = color[c]
    
    images['synthetic'] = img
    print("✓ Created synthetic test image")

print(f"\n✓ Total images loaded: {len(images)}")

## 4. Visualize Sample Images

In [ ]:
# Display all loaded images
n_images = len(images)
fig, axes = plt.subplots(1, n_images, figsize=(5*n_images, 5))

if n_images == 1:
    axes = [axes]

for idx, (name, img) in enumerate(images.items()):
    axes[idx].imshow(img)
    axes[idx].set_title(f'{name.capitalize()} Image\nShape: {img.shape}')
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

## 5. Basic Cellpose Segmentation

Let's run Cellpose on our first image using the 'cyto2' model.

In [ ]:
# Initialize Cellpose model with CPU and smaller batch size
from cellpose import models
import gc  # garbage collection

# Force CPU usage and reduce memory
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'  # Disable GPU

try:
    # Initialize model
    model = models.CellposeModel(model_type='cyto2', gpu=False)
    print("Model loaded successfully on CPU!")
    
    # Get the first image
    img_name = list(images.keys())[0]
    img = images[img_name]
    
    # Resize image if too large
    max_dim = 512
    if max(img.shape[:2]) > max_dim:
        from skimage.transform import resize
        scale = max_dim / max(img.shape[:2])
        new_shape = (int(img.shape[0] * scale), int(img.shape[1] * scale))
        img = resize(img, new_shape, preserve_range=True).astype(np.uint8)
        print(f"Resized image to {new_shape} for memory efficiency")
    
    # Run segmentation with lower batch size
    print(f"\nRunning segmentation on {img_name} image...")
    masks, flows, styles, diams = model.eval(
        img, 
        diameter=None, 
        channels=[0,0],
        batch_size=8,  # Reduce batch size
        resample=False  # Disable resampling to save memory
    )
    
    print(f"✓ Segmentation complete!")
    print(f"  - Number of cells detected: {len(np.unique(masks)) - 1}")
    print(f"  - Estimated cell diameter: {diams:.1f} pixels")
    
    # Clean up memory
    gc.collect()
    
except Exception as e:
    print(f"Error during segmentation: {e}")
    print("\nTrying with minimal settings...")
    
    # Try with even more conservative settings
    try:
        # Resize to smaller size
        from skimage.transform import resize
        small_img = resize(img, (256, 256), preserve_range=True).astype(np.uint8)
        
        # Run with minimal model
        model = models.CellposeModel(model_type='cyto', gpu=False)
        masks, flows, styles, diams = model.eval(
            small_img,
            diameter=30,  # Fixed diameter
            channels=[0,0],
            batch_size=4
        )
        
        print("✓ Segmentation successful with reduced settings")
        print(f"  - Detected {len(np.unique(masks))-1} cells")
        
        # Update variables for subsequent cells
        img = small_img
        
    except Exception as e2:
        print(f"Still failing: {e2}")
        print("Creating dummy results for demonstration...")
        
        # Create dummy masks for visualization
        masks = np.zeros(img.shape[:2], dtype=np.int32)
        flows = [np.zeros((3, img.shape[0], img.shape[1]))]
        diams = 30.0

## 6. Visualize Segmentation Results

In [ ]:
# Create visualization
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Original image
axes[0,0].imshow(img)
axes[0,0].set_title('Original Image')
axes[0,0].axis('off')

# Segmentation masks
axes[0,1].imshow(masks, cmap='tab20')
axes[0,1].set_title(f'Segmentation Masks\n({len(np.unique(masks))-1} cells)')
axes[0,1].axis('off')

# Overlay
overlay = plot.mask_overlay(img, masks)
axes[0,2].imshow(overlay)
axes[0,2].set_title('Overlay')
axes[0,2].axis('off')

# Flow fields
axes[1,0].imshow(flows[0][0], cmap='RdBu_r')
axes[1,0].set_title('Horizontal Flow (X)')
axes[1,0].axis('off')

axes[1,1].imshow(flows[0][1], cmap='RdBu_r')
axes[1,1].set_title('Vertical Flow (Y)')
axes[1,1].axis('off')

# Cell probability
axes[1,2].imshow(flows[0][2], cmap='plasma')
axes[1,2].set_title('Cell Probability')
axes[1,2].axis('off')

plt.tight_layout()
plt.show()

## 7. Compare Different Models

Let's compare the three main Cellpose models: cyto, cyto2, and nuclei.

In [ ]:
# Define models to compare
model_types = ['cyto', 'cyto2', 'nuclei']
results = {}

# Run each model
for model_type in model_types:
    print(f"Running {model_type} model...")
    # Use CellposeModel instead of Cellpose
    model = models.CellposeModel(model_type=model_type)
    masks, flows, styles, diams = model.eval(img, diameter=None, channels=[0,0])
    results[model_type] = {
        'masks': masks,
        'n_cells': len(np.unique(masks)) - 1,
        'diameter': diams
    }
    print(f"  ✓ Detected {results[model_type]['n_cells']} cells")

# Visualize comparisons
fig, axes = plt.subplots(1, 4, figsize=(20, 5))

# Original
axes[0].imshow(img)
axes[0].set_title('Original Image')
axes[0].axis('off')

# Model results
for idx, model_type in enumerate(model_types):
    overlay = plot.mask_overlay(img, results[model_type]['masks'])
    axes[idx+1].imshow(overlay)
    axes[idx+1].set_title(f'{model_type.capitalize()} Model\n{results[model_type]["n_cells"]} cells')
    axes[idx+1].axis('off')

plt.tight_layout()
plt.show()

## 8. Interactive Parameter Exploration

Explore how different parameters affect segmentation results.

In [ ]:
# Create interactive widgets
model_widget = widgets.Dropdown(
    options=['cyto', 'cyto2', 'nuclei'],
    value='cyto2',
    description='Model:'
)

diameter_widget = widgets.IntSlider(
    value=30,
    min=0,
    max=100,
    step=5,
    description='Diameter:',
    tooltip='Set to 0 for automatic estimation'
)

flow_threshold_widget = widgets.FloatSlider(
    value=0.4,
    min=0.0,
    max=3.0,
    step=0.1,
    description='Flow Threshold:'
)

cellprob_threshold_widget = widgets.FloatSlider(
    value=0.0,
    min=-6.0,
    max=6.0,
    step=0.5,
    description='Cell Prob Threshold:'
)

output = widgets.Output()

def update_segmentation(model_type, diameter, flow_threshold, cellprob_threshold):
    with output:
        clear_output(wait=True)
        
        # Run segmentation - use CellposeModel
        model = models.CellposeModel(model_type=model_type)
        diameter_use = None if diameter == 0 else diameter
        
        masks, flows, styles, diams = model.eval(
            img, 
            diameter=diameter_use, 
            channels=[0,0],
            flow_threshold=flow_threshold,
            cellprob_threshold=cellprob_threshold
        )
        
        # Display results
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        
        axes[0].imshow(img)
        axes[0].set_title('Original')
        axes[0].axis('off')
        
        axes[1].imshow(masks, cmap='tab20')
        axes[1].set_title(f'Masks ({len(np.unique(masks))-1} cells)')
        axes[1].axis('off')
        
        overlay = plot.mask_overlay(img, masks)
        axes[2].imshow(overlay)
        axes[2].set_title('Overlay')
        axes[2].axis('off')
        
        plt.tight_layout()
        plt.show()
        
        print(f"Estimated diameter: {diams:.1f} pixels")

# Create interactive interface
interact = widgets.interactive(
    update_segmentation,
    model_type=model_widget,
    diameter=diameter_widget,
    flow_threshold=flow_threshold_widget,
    cellprob_threshold=cellprob_threshold_widget
)

display(interact, output)

## 9. Batch Processing Example

Process all loaded images and create a summary.

In [ ]:
# Process all images
model = models.CellposeModel(model_type='cyto2')  # Use CellposeModel
batch_results = {}

print("Processing all images...\n")

for img_name, img in images.items():
    print(f"Processing {img_name}...")
    masks, flows, styles, diams = model.eval(img, diameter=None, channels=[0,0])
    
    batch_results[img_name] = {
        'masks': masks,
        'n_cells': len(np.unique(masks)) - 1,
        'diameter': diams,
        'image': img
    }
    print(f"  ✓ Found {batch_results[img_name]['n_cells']} cells\n")

# Create summary visualization
n_imgs = len(batch_results)
fig, axes = plt.subplots(2, n_imgs, figsize=(5*n_imgs, 10))

if n_imgs == 1:
    axes = axes.reshape(-1, 1)

for idx, (name, result) in enumerate(batch_results.items()):
    # Original
    axes[0, idx].imshow(result['image'])
    axes[0, idx].set_title(f'{name.capitalize()}\nOriginal')
    axes[0, idx].axis('off')
    
    # Segmentation
    overlay = plot.mask_overlay(result['image'], result['masks'])
    axes[1, idx].imshow(overlay)
    axes[1, idx].set_title(f"{result['n_cells']} cells\nØ {result['diameter']:.1f}px")
    axes[1, idx].axis('off']

plt.tight_layout()
plt.show()

## 10. Export Results

Save segmentation masks for further analysis.

In [ ]:
# Create output directory
os.makedirs('output', exist_ok=True)

# Save masks for each image
for name, result in batch_results.items():
    # Save mask
    mask_filename = f'output/{name}_masks.npy'
    np.save(mask_filename, result['masks'])
    
    # Save overlay visualization
    fig, ax = plt.subplots(figsize=(8, 8))
    overlay = plot.mask_overlay(result['image'], result['masks'])
    ax.imshow(overlay)
    ax.set_title(f"{name.capitalize()} - {result['n_cells']} cells detected")
    ax.axis('off')
    plt.savefig(f'output/{name}_overlay.png', dpi=150, bbox_inches='tight')
    plt.close()
    
    print(f"✓ Saved results for {name}")

print("\nAll results saved to 'output' directory!")

## Summary

In this notebook, we've covered:

1. ✅ Installing Cellpose in a Jupyter environment
2. ✅ Loading and visualizing sample images
3. ✅ Running basic segmentation with Cellpose
4. ✅ Understanding the flow fields and probability maps
5. ✅ Comparing different pre-trained models
6. ✅ Interactive parameter exploration
7. ✅ Batch processing multiple images
8. ✅ Exporting results for further analysis

### Next Steps

- Try with your own images
- Explore Cellpose 2.0's custom training features
- Test Cellpose 3.0's image restoration capabilities
- Integrate with QuPath for whole-slide analysis

### Resources

- [Cellpose Documentation](https://cellpose.readthedocs.io/)
- [Cellpose GitHub](https://github.com/MouseLand/cellpose)
- [Cellpose Paper](https://www.nature.com/articles/s41592-020-01018-x)
- [Interactive Demo](https://www.cellpose.org)